## Memory to LLM Model

##### Memory classes provide memory with chat context in LLM chain
There are different types of Memory classes   
 - **ConversationBufferMemory**: Provided complete chat history in LLM Chain
 - **ConversationBufferWindowMemory**: Provide last few messages as chat history. Reduces the token sent to LLM
 - **COnversationSummaryMemory**: Provides chat summary instead of complete history.
 - **Runnables with History**: These are in new ways to use chat history with Runnables / LCEL

In [1]:
#### Create LLM Model
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

### Conversation Buffer Memory

In [2]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

# Create a memory
memory = ConversationBufferMemory(return_messages = True)

# Create Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are a question answer AI model having conversation with human"),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Create a chain
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, output_parser=StrOutputParser())

# Lets have conversation
while True:
    question = input("Ask your question? ").strip().strip('\n')
    if question.lower() == 'bye' or question.lower() == 'quit' or question.lower() == 'exit':
        print("Bye !!")
        break

    answer = chain.invoke({"question": question})
    print(answer)
    print("")

/tmp/ipykernel_7214/2941381658.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages = True)
/tmp/ipykernel_7214/2941381658.py:24: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt, memory=memory, output_parser=StrOutputParser())


Ask your question?  Which country is biggest exporter of sugarcane


{'question': 'Which country is biggest exporter of sugarcane', 'history': [HumanMessage(content='Which country is biggest exporter of sugarcane', additional_kwargs={}, response_metadata={}), AIMessage(content='The United States is the largest importer of sugar cane. They have a significant percentage of their own production and consumption of sugarcane compared to other countries in North America and Europe, but they are also an important supplier of raw materials for the manufacturing process of sugar.', additional_kwargs={}, response_metadata={})], 'text': 'The United States is the largest importer of sugar cane. They have a significant percentage of their own production and consumption of sugarcane compared to other countries in North America and Europe, but they are also an important supplier of raw materials for the manufacturing process of sugar.'}



Ask your question?  bye


Bye !!


#### Since there is lot of boilerplate we have to do here, there is a specialised chain created to avoid boilerplate code in prompt

In [8]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Create a memory
memory = ConversationBufferMemory(return_messages = True)

# Create a chain
chain = ConversationChain(llm=llm, memory=memory, output_parser=StrOutputParser())

# Lets have conversation
while True:
    question = input("Ask your question? ").strip().strip('\n')
    if question.lower() == 'bye' or question.lower() == 'quit' or question.lower() == 'exit':
        print("Bye !!")
        break

    answer = chain.invoke({"input": question})
    print(answer)
    print("")

Ask your question?  which country is biggest exporter of cotton?


{'input': 'which country is biggest exporter of cotton?', 'history': [HumanMessage(content='which country is biggest exporter of cotton?', additional_kwargs={}, response_metadata={}), AIMessage(content='The United States is the largest exporter of cotton among known countries.', additional_kwargs={}, response_metadata={})], 'response': 'The United States is the largest exporter of cotton among known countries.'}



Ask your question?  What is value of cotton export of this country?


{'input': 'What is value of cotton export of this country?', 'history': [HumanMessage(content='which country is biggest exporter of cotton?', additional_kwargs={}, response_metadata={}), AIMessage(content='The United States is the largest exporter of cotton among known countries.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is value of cotton export of this country?', additional_kwargs={}, response_metadata={}), AIMessage(content="Value of cotton exports to this country is [specific amount or figure]. I don't have access to real-time data, but based on general information, the United States is often considered one of the largest exporters of cotton. The total value of cotton exports from the US to other countries can vary and depends on various factors such as market conditions, global demand patterns, and specific contracts signed by both sides. For the most accurate and up-to-date figures, it would be best to check official government statistics or recent

Ask your question?  bye


Bye !!


### Next is Conversation Buffer Window Memory 
which stores fixed set if memory and if memory overflows, it deletes the old memory

In [4]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

# Create a memory which will remember only last 2 messages 
memory = ConversationBufferWindowMemory(k = 2) 

# Create a chain
chain = ConversationChain(llm=llm, memory=memory, output_parser=StrOutputParser())

# Lets have conversation
while True:
    question = input("Ask your question? ").strip().strip('\n')
    if question.lower() == 'bye' or question.lower() == 'quit' or question.lower() == 'exit':
        print("Bye !!")
        break

    answer = chain.invoke({"input": question})
    print(answer)
    print("")

/tmp/ipykernel_7214/641125888.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k = 2)
/tmp/ipykernel_7214/641125888.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(llm=llm, memory=memory, output_parser=StrOutputParser())


Ask your question?  Which is largest exporter of cotton?


{'input': 'Which is largest exporter of cotton?', 'history': '', 'response': "Without knowing the exact data or information about which country exports the most cotton, I cannot definitively state which country is the largest exporter. The number and volume of exports can vary significantly from year to year and by market and category. If you have specific figures or estimates regarding this topic, I'd be happy to try and provide an answer based on that data rather than general knowledge about global export trends."}



Ask your question?  what is another good most exported by this country?


{'input': 'what is another good most exported by this country?', 'history': "Human: Which is largest exporter of cotton?\nAI: Without knowing the exact data or information about which country exports the most cotton, I cannot definitively state which country is the largest exporter. The number and volume of exports can vary significantly from year to year and by market and category. If you have specific figures or estimates regarding this topic, I'd be happy to try and provide an answer based on that data rather than general knowledge about global export trends.", 'response': "The other notable cotton exporter for the United States was Canada. Specifically:\n\n- The United States imported 109 million metric tons of cotton in 2021, an increase from 75 million metric tons in 2020.\n- Canada was the largest cotton importer among all U.S. exporters that year, accounting for approximately 43% of the total import volume.\n\nHowever, it's important to note that this is based on specific data 

...

{'input': 'Is there any second country which also export this good?', 'history': "Human: Which is largest exporter of cotton?\nAI: Without knowing the exact data or information about which country exports the most cotton, I cannot definitively state which country is the largest exporter. The number and volume of exports can vary significantly from year to year and by market and category. If you have specific figures or estimates regarding this topic, I'd be happy to try and provide an answer based on that data rather than general knowledge about global export trends.\nHuman: what is another good most exported by this country?\nAI: The other notable cotton exporter for the United States was Canada. Specifically:\n\n- The United States imported 109 million metric tons of cotton in 2021, an increase from 75 million metric tons in 2020.\n- Canada was the largest cotton importer among all U.S. exporters that year, accounting for approximately 43% of the total import volume.\n\nHowever, it's

Ask your question?  what is percentage share of this country?


{'input': 'what is percentage share of this country?', 'history': "Human: what is another good most exported by this country?\nAI: The other notable cotton exporter for the United States was Canada. Specifically:\n\n- The United States imported 109 million metric tons of cotton in 2021, an increase from 75 million metric tons in 2020.\n- Canada was the largest cotton importer among all U.S. exporters that year, accounting for approximately 43% of the total import volume.\n\nHowever, it's important to note that this is based on specific data and not a definitive statement about the most exported by any country. The number can fluctuate over time due to factors like market dynamics and economic conditions.\nHuman: Is there any second country which also export this good?\nAI: Yes, Canada is another significant exporter of cotton for the United States. Specifically:\n\n- In 2021, Canada imported approximately 43% of U.S. cotton imports, accounting for about 109 million metric tons.\n- This

Ask your question?  bye


Bye !!


#### As we can see above, this stores the last listory, but the problem in thsi case is, we lose some information
#### In order to fix this, we can use COnversation Summary memory which remembers summary of last conversations instead of whole conversation

In [6]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

# Create a memory which will remember summary of history conversations. This will need an LLM to summarize the history
memory = ConversationSummaryMemory(llm = llm) 

# Create a chain
chain = ConversationChain(llm=llm, memory=memory, output_parser=StrOutputParser())

# Lets have conversation
while True:
    question = input("Ask your question? ").strip().strip('\n')
    if question.lower() == 'bye' or question.lower() == 'quit' or question.lower() == 'exit':
        print("Bye !!")
        break

    answer = chain.invoke({"input": question})
    print(answer)
    print("")

Ask your question?  Answer in 1 word, which country is biggest exporter of cotton?


{'input': 'Answer in 1 word, which country is biggest exporter of cotton?', 'history': '', 'response': 'India'}



Ask your question?  Answer in 1 word, which other good is this country biggest exporter of?


{'input': 'Answer in 1 word, which other good is this country biggest exporter of?', 'history': 'The human asks about the largest cotton exporter. The AI thinks China is the largest cotton exporter due to its extensive cultivation and export capabilities.', 'response': 'China'}



Ask your question?  China vs India who produce and export most sugarcane?


{'input': 'China vs India who produce and export most sugarcane?', 'history': 'AI believes that the country with a large number of cotton growers as exporters is China.', 'response': "India is known for producing and exporting sugar cane, which makes India one of the leading producers and exporters globally. However, it's important to note that India has faced challenges in recent years related to increased demand from other countries due to climate change, rising food prices, and changes in agricultural policies.\n\nChina, on the other hand, is a significant producer of sugarcane and sugar products. China produces around 85% of the world's cane sugar production, which includes some imports. China also has a long history of sugarcane cultivation and processing, making it an important player in global trade and supply chains.\n\nThe question about who produce and export most sugarcane is therefore somewhat nuanced:\n\n1. **China (production and exports)**: Produces around 85% of the wor

Ask your question?  bye


Bye !!


## Runnables
Conversation chain with memory has been deprecated in Langchain and Runnables are way to go forward for Langchain.

In [31]:
# Lets say we have an LLM Chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate)
from langchain.schema import HumanMessage, SystemMessage

prompt = ChatPromptTemplate.from_messages([
    SystemMessage("Provide the answer to the user question in 3-5 words only."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("Question: {question}")
])

chain = prompt | llm | StrOutputParser()

# Test
chain.invoke({"question": "Which country is largest producer of cotton?", "history": []})

'USA'

In [32]:
# Lets integrate Memory in this chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# This object will store the history. This can also be a database
store = {}

# This method defined for given session id, how to get history from store
def get_hist_for_session(session: str) -> BaseChatMessageHistory:
    if session not in store.keys():
        store[session] = ChatMessageHistory()

    return store[session]

# Create runnable
runnable = RunnableWithMessageHistory(
    chain,
    get_hist_for_session,
    input_messages_key="question",
    history_messages_key="history"
)

# Invoke the runnable
answer = runnable.invoke({"question": "Which country is largest producer of cotton?"}, 
                         config={"configurable": {"session_id": "ses1"}})
print(answer)
# If we see the store now
print(store)

USA
{'ses1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Which country is largest producer of cotton?', additional_kwargs={}, response_metadata={}), AIMessage(content='USA', additional_kwargs={}, response_metadata={})])}


In [33]:
# If we ask follow up question with same session ID, we will get correct response
answer = runnable.invoke({"question": "What is the per capita income of this country?"}, 
                         config={"configurable": {"session_id": "ses1"}})
print(answer)

$21,679 USD/yr


In [34]:
# But If we give different session id, it will not know which country we are talking about
answer = runnable.invoke({"question": "What is the per capita income of this country?"}, 
                         config={"configurable": {"session_id": "ses2"}})
print(answer)

Not specified
